In [1]:
include("prepare_data_Sioux_simp.jl")

furInfo (generic function with 1 method)

In [2]:
demandsDict = Dict{Int64, Dict{(Int64,Int64), Float64}}()
# get ground trueth demands, indexed by 0
demandsDict[0] = iniDemand("../data_original/SiouxFalls_trips_simp.txt")
# get initial demands, indexed by 1
demandsDict[1] = iniDemand("../data_original/SiouxFalls_trips_simp.txt", 1)

numNodes, numLinks, numODpairs, capacity, free_flow_time, ta_data = paraNetwork("Sioux_simp")
numRoutes, odPairRoute, linkRoute, link_label_dict = furInfo()

(636,["100-471"=>1,"51-242"=>1,"56-264"=>1,"69-326"=>1,"46-220"=>1,"66-313"=>1,"79-362"=>1,"121-574"=>1,"7-23"=>1,"19-81"=>1,"103-487"=>1,"13-54"=>1,"20-86"=>1,"44-211"=>1,"31-148"=>1,"126-597"=>1,"130-629"=>1,"107-504"=>1,"111-520"=>1,"95-446"=>1,"78-357"=>1,"43-204"=>1,"26-113"=>1,"107-505"=>1,"61-289"=>1,"9-39"=>1,"110-518"=>1,"3-7"=>1,"127-604"=>1,"63-294"=>1,"122-577"=>1,"37-173"=>1,"96-448"=>1,"94-441"=>1,"52-244"=>1,"131-632"=>1,"106-501"=>1,"23-101"=>1,"105-494"=>1,"103-485"=>1,"102-484"=>1,"128-611"=>1,"95-445"=>1,"84-387"=>1,"14-58"=>1,"118-561"=>1,"14-60"=>1,"117-557"=>1,"53-251"=>1,"34-159"=>1,"76-346"=>1,"31-150"=>1,"124-587"=>1,"57-268"=>1,"19-79"=>1,"81-375"=>1,"48-228"=>1,"55-258"=>1,"18-74"=>1,"67-320"=>1,"108-508"=>1,"63-296"=>1,"8-31"=>1,"113-533"=>1,"33-156"=>1,"42-202"=>1,"78-356"=>1,"129-615"=>1,"45-212"=>1,"112-526"=>1,"91-424"=>1,"9-40"=>1,"120-570"=>1,"109-512"=>1,"118-559"=>1,"100-466"=>1,"42-203"=>1,"92-431"=>1,"20-87"=>1,"86-392"=>1,"74-340"=>1,"97-450"=>1,"

In [3]:
using JuMP

function TAP(demands, fcoeffs)

    #load OD pair labels
    odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json")
    odPairLabel = JSON.parse(odPairLabel)

    odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json")
    odPairLabel_ = JSON.parse(odPairLabel_)

    #load link labels
    linkLabel = readall("link_label_dict_Sioux_simp.json")
    linkLabel = JSON.parse(linkLabel)

    linkLabel_ = readall("link_label_dict_Sioux_simp_.json")
    linkLabel_ = JSON.parse(linkLabel_)

    #load node-link incidence
    nodeLink = readall("node_link_incidence_Sioux_simp.json")
    nodeLink = JSON.parse(nodeLink)

    start_node = ta_data.start_node

    demandsVec = zeros(length(odPairLabel_))

    for i = 1:length(demandsVec)
        demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
    end

    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    @defVar(m, linkFlow[1:numLinks])

    @defVar(m, odLinkFlow[keys(demands), 1:numLinks])
    for (s,t) = keys(demands)
        for k = 1:numLinks
            @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
        end
    end

    for k = 1:numLinks
        linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
        @addConstraint(m, linkFlowPartial == linkFlow[k])
    end

    for l = 1:numNodes
        for (s,t) = keys(demands)
            if s != t
                odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
                if (l == s)
                    @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
                elseif (l == t)
                    @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
                else
                    @addConstraint(m, odLinkFlowPartial == 0)
                end
            end
        end
    end
    
    @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), 
        a = 1:numLinks})

    @setNLObjective(m, Min, f)

    TT = STDOUT # save original STDOUT stream
    redirect_stdout()
    solve(m)
    redirect_stdout(TT) # restore STDOUT

    tapFlowVect = getValue(linkFlow)

    tapFlows = Dict{(Int64,Int64),Float64}()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        tapFlows[key] = tapFlowVect[i]
    end

#    getObjectiveValue(m)

    return tapFlows, tapFlowVect
end

TAP (generic function with 1 method)

In [4]:
# include("tap_Sioux_simp.jl")

In [5]:
fcoeffs = [1, 0, 0, 0, .15]

5-element Array{Float64,1}:
 1.0 
 0.0 
 0.0 
 0.0 
 0.15

In [6]:
tapFlowVecDict = Dict{Int64, Array{Float64, 1}}()
tapFlowDicDict = Dict{Int64, Dict{(Int64,Int64),Float64}}()

Dict{Int64,Dict{(Int64,Int64),Float64}} with 0 entries

# generate ground trueth flow vector, indexed by 0 (i.e., the observed flow vector $\tilde {\textbf{x}}$

In [7]:
tapFlowDicDict[0], tapFlowVecDict[0] = TAP(demandsDict[0], fcoeffs)

([(11,12)=>4194.728975972957,(4,3)=>7105.2710212288885,(2,6)=>3499.9999986713683,(11,4)=>4155.548193381439,(5,6)=>8050.27717071561,(4,11)=>4202.510698517833,(8,6)=>7541.964082860455,(5,4)=>10641.964082937508,(10,11)=>8158.035914355255,(2,1)=>2299.999998663629,(8,9)=>5158.035914394822,(12,3)=>2994.7289762235932,(11,10)=>8249.722826326752,(3,1)=>3899.999998689427,(5,9)=>7899.9999987258425,(6,2)=>3499.9999986701755,(6,8)=>7550.277170840937,(4,5)=>10650.277170964806,(1,3)=>3899.999998688233,(8,7)=>6499.999998663742,(7,8)=>6499.999998663742,(12,11)=>4239.453382808038,(9,10)=>13758.035914328017,(3,12)=>3039.4533830586906,(6,5)=>8041.964082736314,(1,2)=>2299.9999986648204,(9,8)=>5149.722826414328,(10,9)=>13749.722826299527,(3,4)=>7060.546614392599,(9,5)=>7899.999998677847],[2300.0,3900.0,2300.0,3500.0,3900.0,7060.55,3039.45,7105.27,10650.3,4202.51  …  7900.0,5149.72,13758.0,13749.7,8158.04,4155.55,8249.72,4194.73,2994.73,4239.45])

# generate initial flow vector, indexed by 1 (i.e., the flow vector $\textbf{x}^0$)

In [8]:
tapFlowDicDict[1], tapFlowVecDict[1] = TAP(demandsDict[1], fcoeffs)

([(11,12)=>4230.34621539922,(4,3)=>7218.668727254906,(2,6)=>3536.512018300016,(11,4)=>4189.59906021842,(5,6)=>7945.688551823705,(4,11)=>4141.495267762386,(8,6)=>7637.345949047434,(5,4)=>10731.06051087691,(10,11)=>7844.7990610199495,(2,1)=>2290.0018232027282,(8,9)=>5302.568177256391,(12,3)=>2973.983955226968,(11,10)=>8159.919584208543,(3,1)=>3983.683171653966,(5,9)=>7848.624947163285,(6,2)=>3477.242068113701,(6,8)=>7605.914939460251,(4,5)=>10542.610381834173,(1,3)=>3858.632081255275,(8,7)=>6564.237480928,(7,8)=>6618.734386556513,(12,11)=>4179.126365550552,(9,10)=>13975.743894559911,(3,12)=>3026.926957577047,(6,5)=>8094.158786485401,(1,2)=>2287.437057806176,(9,8)=>5039.442631788451,(10,9)=>13744.950321872531,(3,4)=>7016.755139818883,(9,5)=>7959.423986541879],[2287.44,3858.63,2290.0,3536.51,3983.68,7016.76,3026.93,7218.67,10542.6,4141.5  …  7959.42,5039.44,13975.7,13745.0,7844.8,4189.6,8159.92,4230.35,2973.98,4179.13])

In [9]:
tapFlowDicDict[0]

Dict{(Int64,Int64),Float64} with 30 entries:
  (11,12) => 4194.728975972957
  (4,3)   => 7105.2710212288885
  (2,6)   => 3499.9999986713683
  (11,4)  => 4155.548193381439
  (5,6)   => 8050.27717071561
  (4,11)  => 4202.510698517833
  (8,6)   => 7541.964082860455
  (5,4)   => 10641.964082937508
  (10,11) => 8158.035914355255
  (2,1)   => 2299.999998663629
  (8,9)   => 5158.035914394822
  (12,3)  => 2994.7289762235932
  (11,10) => 8249.722826326752
  (3,1)   => 3899.999998689427
  (5,9)   => 7899.9999987258425
  (6,2)   => 3499.9999986701755
  (6,8)   => 7550.277170840937
  (4,5)   => 10650.277170964806
  (1,3)   => 3899.999998688233
  (8,7)   => 6499.999998663742
  (7,8)   => 6499.999998663742
  (12,11) => 4239.453382808038
  (9,10)  => 13758.035914328017
  (3,12)  => 3039.4533830586906
  (6,5)   => 8041.964082736314
  ⋮       => ⋮

In [10]:
tapFlowDicDict[1]

Dict{(Int64,Int64),Float64} with 30 entries:
  (11,12) => 4230.34621539922
  (4,3)   => 7218.668727254906
  (2,6)   => 3536.512018300016
  (11,4)  => 4189.59906021842
  (5,6)   => 7945.688551823705
  (4,11)  => 4141.495267762386
  (8,6)   => 7637.345949047434
  (5,4)   => 10731.06051087691
  (10,11) => 7844.7990610199495
  (2,1)   => 2290.0018232027282
  (8,9)   => 5302.568177256391
  (12,3)  => 2973.983955226968
  (11,10) => 8159.919584208543
  (3,1)   => 3983.683171653966
  (5,9)   => 7848.624947163285
  (6,2)   => 3477.242068113701
  (6,8)   => 7605.914939460251
  (4,5)   => 10542.610381834173
  (1,3)   => 3858.632081255275
  (8,7)   => 6564.237480928
  (7,8)   => 6618.734386556513
  (12,11) => 4179.126365550552
  (9,10)  => 13975.743894559911
  (3,12)  => 3026.926957577047
  (6,5)   => 8094.158786485401
  ⋮       => ⋮

In [11]:
outfile = open("tapFlows.json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [12]:
tapFlowDicDict_ = readall("tapFlows.json")
tapFlowDicDict_ = JSON.parse(tapFlowDicDict_)

Dict{UTF8String,Any} with 2 entries:
  "1" => ["(9,10)"=>13975.743894559911,"(8,7)"=>6564.237480928,"(6,8)"=>7605.91…
  "0" => ["(9,10)"=>13758.035914328017,"(8,7)"=>6499.999998663742,"(6,8)"=>7550…

In [13]:
tapFlowVecDict[0]

30-element Array{Float64,1}:
  2300.0 
  3900.0 
  2300.0 
  3500.0 
  3900.0 
  7060.55
  3039.45
  7105.27
 10650.3 
  4202.51
 10642.0 
  8050.28
  7900.0 
     ⋮   
  6500.0 
  5158.04
  7900.0 
  5149.72
 13758.0 
 13749.7 
  8158.04
  4155.55
  8249.72
  4194.73
  2994.73
  4239.45